In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.utils import *
from src.score import *
from src.data_generator import *
from src.networks import *
import matplotlib.pyplot as plt
import yaml

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"]=str(7)
limit_mem()

In [4]:
exp_id = '13-resnet_bn_dr_0.1'
# datadir = '/data/stephan/WeatherBench/2.8125deg/'
datadir = '/data/stephan/WeatherBench/5.625deg/'
model_save_dir = '/home/rasp/data/myWeatherBench/predictions/saved_models/'
pred_save_dir = '/home/rasp/data/myWeatherBench/predictions/'
var_dict = {'geopotential': ('z', [200, 500, 850]), 'temperature': ('t', [200, 500, 850]), 'u_component_of_wind': ('u', [200, 500, 850]), 'v_component_of_wind': ('v', [200, 500, 850]), 'constants': ['lsm','orography','lat2d']}
output_vars = ['z_500', 't_850']
filters = [128, 128, 128, 128, 128, 128, 128, 128, 2]
kernels = [7, 3, 3, 3, 3, 3, 3, 3, 3]
lead_time = 72
lr = 0.5e-4
early_stopping_patience = 10
reduce_lr_patience = 5
data_subsample = 2
norm_subsample = 30000
bn_position = 'post'
use_bias = False
dropout = 0.1

batch_size = 32
nt_in = 1
l2 = 0
skip = True

In [5]:
ds = xr.merge([xr.open_mfdataset(f'{datadir}/{var}/*.nc', combine='by_coords') for var in var_dict.keys()])

In [6]:
ds_train = ds.sel(time=slice('2015', '2015'))
ds_valid = ds.sel(time=slice('2016', '2016'))
ds_test = ds.sel(time=slice('2017', '2018'))

In [7]:
dg_train = DataGenerator(
    ds_train, var_dict, lead_time, batch_size=batch_size, output_vars=output_vars,
    data_subsample=data_subsample, norm_subsample=norm_subsample
)
dg_valid = DataGenerator(
    ds_valid, var_dict, lead_time, batch_size=batch_size, mean=dg_train.mean, std=dg_train.std,
    shuffle=False, output_vars=output_vars
)
dg_test =  DataGenerator(
    ds_test, var_dict, lead_time, batch_size=batch_size, mean=dg_train.mean, std=dg_train.std,
    shuffle=False, output_vars=output_vars
)

DG start 14:08:57.260237
DG normalize 14:08:57.282564
DG load 14:09:00.490292
Loading data into RAM
DG done 14:09:03.372915
DG start 14:09:03.373235
DG normalize 14:09:03.404124
DG load 14:09:03.412138
Loading data into RAM
DG done 14:09:07.587957
DG start 14:09:07.589085
DG normalize 14:09:07.622331
DG load 14:09:07.633060
Loading data into RAM
DG done 14:09:15.329595


In [10]:
X, y = dg_train[0]; X.shape

(32, 32, 64, 15)

In [13]:
model = keras.models.load_model(f'{model_save_dir}/{exp_id}.h5', 
                                custom_objects={'PeriodicConv2D': PeriodicConv2D})

In [14]:
pred = model(X)

In [18]:
keras.losses.mse(y, pred).numpy().mean()

0.028542088

In [32]:
def create_lat_mse(lat):
    weights_lat = np.cos(np.deg2rad(lat)).values
    weights_lat /= weights_lat.mean()
    def lat_mse(y_true, y_pred):
        error = y_true - y_pred
        mse = ((error)**2 * weights_lat[None, : , None, None])
        return mse
    return lat_mse

In [33]:
lmse = create_lat_mse(dg_train.data.lat)

In [36]:
lmse(y, pred).numpy().mean()

0.021703219

In [39]:
model.compile(keras.optimizers.Adam(lr), lmse, metrics=['mse'])

In [40]:
history = model.fit(dg_train, epochs=1, validation_data=dg_valid)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 136 steps, validate for 273 steps
136/136 [==============================] - 28s 203ms/step - loss: 0.0189 - mse: 0.0257 - val_loss: 0.0233 - val_mse: 0.0337
